# LAB 7: Error analysis

Objectives
* Construct a  linear text classifier using SGDClassifier
* Evaluate its performance and categorize the errors that it makes
* Eaxmine model's coefficients and decision function values
* Interpret model results using LIME

In [ ]:
import numpy as np
import pandas as pd
from cytoolz import *
from tqdm.auto import tqdm

tqdm.pandas()

---

## Load data

In [ ]:
train = pd.read_parquet("s3://ling583/lab7-train.parquet", storage_options={"anon":True})
test = pd.read_parquet("s3://ling583/lab7-test.parquet", storage_options={"anon":True})

In [ ]:
import cloudpickle
from sklearn.metrics import classification_report, f1_score, plot_confusion_matrix

In [ ]:
sgd = cloudpickle.load(open("sgd.model", "rb"))

In [ ]:
predicted = sgd.predict(test["text"])
print(classification_report(test["topics"], predicted))

---

## LIME

In [ ]:
from lime.lime_text import LimeTextExplainer
import matplotlib.pyplot as plt

In [ ]:
explainer = LimeTextExplainer(class_names=sgd.named_steps['sgdclassifier'].classes_)

In [ ]:
test["topics"].iloc[1613], test["text"].iloc[1613]

In [ ]:
exp = explainer.explain_instance(
    test["text"].loc[1613], sgd.decision_function, num_features=15, labels=[0, 1, 2, 3]
)

In [ ]:
for i in range(0, 4):
    exp.as_pyplot_figure(i)
plt.plot()

In [ ]:
err = test[(test['topics']=='GVIO')&(predicted=='GPOL')]

In [ ]:
err.iloc[30]

In [ ]:
err.iloc[30]['text']

In [ ]:
exp = explainer.explain_instance(
    err["text"].iloc[30], sgd.decision_function, num_features=15, labels=[1, 3]
)
for i in [1, 3]:
    exp.as_pyplot_figure(i)
plt.plot()

In [ ]:
err.iloc[50]

In [ ]:
err.iloc[50]['text']

In [ ]:
exp = explainer.explain_instance(
    err["text"].iloc[50], sgd.decision_function, num_features=15, labels=[1, 3]
)
for i in [1, 3]:
    exp.as_pyplot_figure(i)
plt.plot()

In [ ]:
import kwic

In [ ]:
kwic.kwic('upper', train['text'])

**TO DO:** Write a brief general summary of the results of this analysis (along with the previous ones). Do we have a good classifier? What would be the next steps for improving it?